<a href="https://colab.research.google.com/github/bhuiyanmobasshir94/MultiCoNER/blob/main/notebooks/spark_nlp_multiconer_000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-01-17 06:14:42--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-01-17 06:14:44--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-01-17 06:14:45--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:44

In [2]:
# !pip install gdown
# !gdown --id 1AaaFcJ0Zzxr-fnlhlbp8mQePU6o8dFvk
# !unzip -u "./BN_DATASET.zip" -d "./data"
# !rm -f "./BN_DATASET.zip"
# !cp -r ./data/BN-Bangla/. ./data && rm -rf ./data/BN-Bangla

In [3]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

In [4]:
# if you have GPU
spark = sparknlp.start(gpu=True)
# spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.4.0
Apache Spark version:  3.0.3


In [5]:
with open("/content/drive/MyDrive/MULTICONER/bn_train.conll") as f:
    c=f.read()

print (c[:200])

# id 5e80f5c9-1196-4906-826b-4cbdcaec7b6f	domain=train
২০১৮ _ _ O
এর _ _ O
সেরা _ _ O
(বর্ণানুক্রমিকভাবে _ _ O
তালিকাভুক্ত, _ _ O
র‍্যাঙ্ক _ _ O
করা _ _ O
হয়নি), _ _ O
এনপিআর _ _ B-CORP

# id ba55278


In [6]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, "/content/drive/MyDrive/MULTICONER/bn_train.conll")
training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|২০১৮ এর সেরা (বর্...|[[document, 0, 71...|[[document, 0, 71...|[[token, 0, 3, ২০...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
|১৮২৫ সালে, তিনি ব...|[[document, 0, 81...|[[document, 0, 81...|[[token, 0, 3, ১৮...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
|তিনি মার্কো মারুল...|[[document, 0, 53...|[[document, 0, 53...|[[token, 0, 3, তি...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [7]:
test_data = CoNLL().readDataset(spark, "/content/drive/MyDrive/MULTICONER/bn_dev.conll")
test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [8]:
training_data.count()

15300

In [9]:
test_data.count()

800

In [10]:
# we use BERT Tiny
bert_annotator = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
.setBatchSize(8)

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


In [11]:
test_data = bert_annotator.transform(test_data)
test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|                bert|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|[[word_embeddings...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|[[word_embeddings...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|[[word_embeddings...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [12]:
# let's transform and save our test dataset for evaluation
test_data.write.parquet("test_withEmbeds.parquet")

In [13]:
test_data.select("bert.result","bert.embeddings",'label.result').show()

+--------------------+--------------------+--------------------+
|              result|          embeddings|              result|
+--------------------+--------------------+--------------------+
|[সমস্ত, বেতন, নিল...|[[-1.1855663, 0.7...|[O, O, O, O, O, O...|
|[সংস্করণ, ৩, প্রক...|[[-1.376434, 0.92...|[O, O, O, O, B-CW...|
|[একটি, পাইলট, (হা...|[[-0.86345154, 0....|[O, O, O, O, O, B...|
|[প্রাইমাভেরা, ঝুঁ...|[[-1.2236066, 0.9...|[B-CW, O, O, O, O...|
|[১৮০০, -এর, দশকে,...|[[-1.1398506, 0.8...|[O, O, O, B-PER, ...|
|[চারজন, বিজয়ী, ড...|[[-1.0752248, 0.2...|[O, O, O, O, B-PE...|
|[এটি, ছিল, নাটকীয...|[[-0.7776178, 0.5...|[O, O, O, O, O, B...|
|[এর, মধ্যে, একটি,...|[[-1.2268137, 0.3...|[O, O, O, B-CORP,...|
|[এরলং, সুযোগ, সমা...|[[-0.9119878, 0.4...|[B-CW, O, O, O, O...|
|[যদিও, গির্জার, স...|[[-1.0261633, 0.6...|[O, O, O, O, B-PR...|
|[দ্য, ম্যাগনিফিসে...|[[-1.5631484, 0.9...|[B-CW, I-CW, I-CW...|
|[দীর্ঘ, সময়, ধরে...|[[-1.3828299, 0.9...|[O, O, O, O, O, O...|
|[তা, সত্ত্বেও,, এ...|[[-

In [14]:
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "bert"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setGraphFolder("/content/drive/MyDrive/MULTICONER")\
  .setMaxEpochs(10)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setTestDataset("test_withEmbeds.parquet")

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

In [15]:
ner_model = pipeline.fit(training_data)

In [16]:
!ls -l /root/annotator_logs/

total 12
-rw-r--r-- 1 root root 10120 Jan 17 06:28 NerDLApproach_2f8732b6f85a.log


In [17]:
!cat /root/annotator_logs/NerDLApproach_2f8732b6f85a.log

Name of the selected graph: /content/drive/MyDrive/MULTICONER/blstm_13_128_128_220.pb
Training started - total epochs: 10 - lr: 0.001 - batch size: 32 - labels: 13 - chars: 219 - training examples: 15300


Epoch 1/10 started, lr: 0.001, dataset size: 15300


Epoch 1/10 - 51.24s - loss: 4078.211 - batches: 480
Quality on test dataset: 
time to finish evaluation: 1.71s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 19	 39	 82	 0.3275862	 0.18811882	 0.23899372
B-PROD	 4	 16	 186	 0.2	 0.021052632	 0.03809524
B-CW	 0	 0	 120	 0.0	 0.0	 0.0
I-GRP	 103	 167	 123	 0.38148147	 0.45575222	 0.4153226
B-CORP	 9	 10	 118	 0.47368422	 0.070866145	 0.12328768
B-GRP	 38	 67	 80	 0.36190477	 0.3220339	 0.3408072
I-CORP	 9	 19	 113	 0.32142857	 0.07377049	 0.120000005
I-LOC	 2	 6	 59	 0.25	 0.032786883	 0.05797101
I-PER	 81	 146	 99	 0.35682818	 0.45	 0.3980344
I-PROD	 2	 17	 127	 0.10526316	 0.015503876	 0.027027026
I-CW	 0	 0	 161	 0.0	 0.0	 0.0
B-PER	 67	 112	 77	 0.37430167	 0.4652778	 0.41486067
tp: 334

In [18]:
ner_model.stages[1].write().overwrite().save('./NER_bert_20200219')

In [19]:
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|                bert|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|[[word_embeddings...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|[[word_embeddings...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|[[word_embeddings...|
|প্রাইমাভেরা ঝুঁকি...|[[document, 0, 64...|[[document, 0, 64...|[[token, 0, 10, প...|[[pos, 0, 10, _, ...|

In [20]:
# let's only feed sentence and token from our test dataset
predictions = ner_model.transform(test_data.select("sentence", "token", "label"))
predictions.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|            sentence|               token|               label|                bert|                 ner|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[[document, 0, 10...|[[token, 0, 4, সম...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|[[document, 0, 16...|[[token, 0, 6, সং...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|[[document, 0, 11...|[[token, 0, 3, এক...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [21]:
predictions.select('token.result','label.result','ner.result').show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+
|                                  result|                                  result|                                  result|
+----------------------------------------+----------------------------------------+----------------------------------------+
|[সমস্ত, বেতন, নিলামের, সাধারণ, ব্যবহা...|[O, O, O, O, O, O, O, O, O, O, B-CORP...|[O, O, O, O, O, O, O, O, O, O, O, O, ...|
|[সংস্করণ, ৩, প্রকাশের, আগে,, গ্রাফিক্...|[O, O, O, O, B-CW, I-CW, I-CW, O, O, ...|[O, O, O, O, B-GRP, I-GRP, I-GRP, I-G...|
|[একটি, পাইলট, (হাটন, দ্বারা, চিত্রিত)...|[O, O, O, O, O, B-LOC, I-LOC, O, O, O...|[O, O, O, O, O, B-PROD, I-PROD, O, O,...|
|[প্রাইমাভেরা, ঝুঁকি, বিশ্লেষণ, -, প্র...|          [B-CW, O, O, O, O, O, O, O, O]|             [O, O, O, O, O, O, O, O, O]|
|[১৮০০, -এর, দশকে, ম্যাটভে, কাজাকভ, দ্...|[O, O, O, B-PER, I-PER, O, O, O, O, O...|[O, O, O, B-PER, I-PER, O, O, O, O, O...|


In [22]:
predictions.printSchema()

root
 |-- sentence: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (n

In [23]:
import pyspark.sql.functions as F

predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

+----------+------------+----------+
|token     |ground_truth|prediction|
+----------+------------+----------+
|সমস্ত     |O           |O         |
|বেতন      |O           |O         |
|নিলামের   |O           |O         |
|সাধারণ    |O           |O         |
|ব্যবহারিক |O           |O         |
|উদাহরণ    |O           |O         |
|বিভিন্ন   |O           |O         |
|পেনি      |O           |O         |
|নিলাম     |O           |O         |
|/         |O           |O         |
|বিডিং     |B-CORP      |O         |
|ফি        |I-CORP      |O         |
|নিলাম     |I-CORP      |O         |
|ওয়েবসাইটে|O           |O         |
|পাওয়া    |O           |O         |
|যাবে।     |O           |O         |
|সংস্করণ   |O           |O         |
|৩         |O           |O         |
|প্রকাশের  |O           |O         |
|আগে,      |O           |O         |
+----------+------------+----------+
only showing top 20 rows



In [24]:
import pandas as pd

df = predictions.select('token.result','label.result','ner.result').toPandas()

df

,result,result,result
0,"[সমস্ত, বেতন, নিলামের, সাধারণ, ব্যবহারিক, উদাহ...","[O, O, O, O, O, O, O, O, O, O, B-CORP, I-CORP,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[সংস্করণ, ৩, প্রকাশের, আগে,, গ্রাফিক্যাল, ইউজা...","[O, O, O, O, B-CW, I-CW, I-CW, O, O, O, O, O, ...","[O, O, O, O, B-GRP, I-GRP, I-GRP, I-GRP, I-GRP..."
2,"[একটি, পাইলট, (হাটন, দ্বারা, চিত্রিত), লস, অ্য...","[O, O, O, O, O, B-LOC, I-LOC, O, O, O, O, O, O...","[O, O, O, O, O, B-PROD, I-PROD, O, O, O, O, O,..."
3,"[প্রাইমাভেরা, ঝুঁকি, বিশ্লেষণ, -, প্রকল্প, ঝুঁ...","[B-CW, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O]"
4,"[১৮০০, -এর, দশকে, ম্যাটভে, কাজাকভ, দ্বারা, ডিজ...","[O, O, O, B-PER, I-PER, O, O, O, O, O, O, O, O...","[O, O, O, B-PER, I-PER, O, O, O, O, O, O, O, O..."
...,...,...,...
795,"[বাসাটি, পাতালরেল, এ, স্টেশন, হিসেবে, ব্যবহৃত,...","[O, B-GRP, O, O, O, O, O]","[O, B-LOC, O, O, O, O, O]"
796,"[১৯৮৫, সালে, বন্ধ, হওয়ার, আগে,, ডিপোতে, ব্রিট...","[O, O, O, O, O, O, B-PROD, I-PROD, I-PROD, I-P...","[O, O, O, O, O, B-GRP, I-GRP, I-GRP, I-GRP, O,..."
797,"[প্রথম, মৌসুমটি, সমালোচকদের, কাছ, থেকে, সাধারন...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
798,"[বিক্রয়, সাধারণত, প্রথম, প্যানেলিস্টের, পরিচয...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [25]:
from pyspark.ml import Pipeline

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(True)

loaded_ner_model = NerDLModel.load("NER_bert_20200219")\
 .setInputCols(["sentence", "token", "bert"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        bert,
        loaded_ner_model,
        converter])

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


In [39]:
empty_data = spark.createDataFrame([['']]).toDF("text")
empty_data.show()

+----+
|text|
+----+
|    |
+----+



In [40]:
prediction_model = ner_prediction_pipeline.fit(empty_data)

In [41]:
text = "Peter Parker is a nice guy and lives in New York."
sample_data = spark.createDataFrame([[text]]).toDF("text")
sample_data.show()

+--------------------+
|                text|
+--------------------+
|Peter Parker is a...|
+--------------------+



In [42]:
preds = prediction_model.transform(sample_data)
preds.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                bert|                 ner|            ner_span|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter Parker is a...|[[document, 0, 48...|[[document, 0, 48...|[[token, 0, 4, Pe...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 11, P...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [43]:
preds.select('ner_span.result').take(1)

[Row(result=['Peter Parker'])]

In [44]:
preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

+------------+------+
|chunk       |entity|
+------------+------+
|Peter Parker|CW    |
+------------+------+



In [32]:
# document = DocumentAssembler()\
#     .setInputCol("text")\
#     .setOutputCol("document")

# sentence = SentenceDetector()\
#     .setInputCols(['document'])\
#     .setOutputCol('sentence')

# token = Tokenizer()\
#     .setInputCols(['sentence'])\
#     .setOutputCol('token')

# loaded_ner_model = NerDLModel.load("NER_bert_20200219")\
#  .setInputCols(["sentence", "token", "glove"])\
#  .setOutputCol("ner")

# converter = NerConverter()\
#   .setInputCols(["document", "token", "ner"])\
#   .setOutputCol("ner_span")

# glove_ner_prediction_pipeline = Pipeline(
#     stages = [
#         document,
#         sentence,
#         token,
#         glove,
#         loaded_ner_model,
#         converter])

NameError: ignored

In [ ]:
# !git clone https://github.com/JohnSnowLabs/spark-nlp

In [ ]:
# %cd spark-nlp/python/tensorflow
# %cd /content

In [ ]:
# %env PYTHONPATH="/env/python:/content/spark-nlp/python/tensorflow/lib/ner"
# %env PYTHONPATH="$/env/python:/content/spark-nlp/python/tensorflow/lib/ner"
# %set_env PYTHONPATH=/content/spark-nlp/python/tensorflow/lib/ner:/env/python ## working solution
# %set_env PYTHONPATH=/env/python

In [ ]:
# !source /content/ner_venv/bin/activate; python ner/create_models.py 13 128 220 /content/data

In [ ]:
# !pip install virtualenv
# !virtualenv ner_venv
# !virtualenv -p /usr/bin/python3.6 ner_venv ## working solution

In [ ]:
# !source /content/ner_venv/bin/activate; pip install tensorflow==1.15.0

Ref: [link](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/blogposts/3.NER_with_BERT.ipynb)